In [1]:
import spacy
from spacy.en import English

In [62]:
!pwd

/Users/bilindhajer/Desktop/atl-dat2/lessons/lesson-14/code/starter-code


In [19]:
def structure_file(string):
    string_split = string.split("|") # list of length 4
    
    first_name = string_split[0]
    last_name  = string_split[1]
    
    age = int(string_split[2])
    
    emails = string_split[3].split(",")
    email_1 = emails[0]
    email_2 = emails[1]
    emails = [email_1,email_2]
    
    final_obj = { 
        "first_name" : first_name,
        "last_name"  : last_name,
        "age" : age, 
        "emails" : emails
    }
    
    return final_obj
    

In [20]:
sentence_1 = "Bilind|Hajer|27|bilind07@gmail.com,bhajer3@gatech.edu"
sentence_2 = "John|Smith|24|JS12@gmail.com,jSmith3@gatech.edu"
sentence_3 = "Sarah|Adams|22|sarah22@gmail.com,sAdams3@gatech.edu"
sentence_4 = "Rob|Adams|24|rob24@gmail.com,rAdams3@gatech.edu"
textfile = [sentence_1,sentence_2,sentence_3,sentence_4]

In [21]:
textfile_structured = map(structure_file,textfile)

In [22]:
for obj in textfile_structured:
    print(obj)

{'first_name': 'Bilind', 'last_name': 'Hajer', 'age': 27, 'emails': ['bilind07@gmail.com', 'bhajer3@gatech.edu']}
{'first_name': 'John', 'last_name': 'Smith', 'age': 24, 'emails': ['JS12@gmail.com', 'jSmith3@gatech.edu']}
{'first_name': 'Sarah', 'last_name': 'Adams', 'age': 22, 'emails': ['sarah22@gmail.com', 'sAdams3@gatech.edu']}
{'first_name': 'Rob', 'last_name': 'Adams', 'age': 24, 'emails': ['rob24@gmail.com', 'rAdams3@gatech.edu']}


In [23]:
# give me only student's bilind's record
student = filter(
    lambda obj: obj["first_name"].lower() == "bilind",
    textfile_structured
)

In [25]:
print(student)

[{'first_name': 'Bilind', 'last_name': 'Hajer', 'age': 27, 'emails': ['bilind07@gmail.com', 'bhajer3@gatech.edu']}]


In [ ]:
# get all students atleast 24 years old

In [38]:
new_sentence = "Hi, my name is Bilind, I like to run, jog, and eat"

In [39]:
sentence_to_unicode = unicode(new_sentence, errors='ignore')

In [40]:
print(setenence_to_unicode)

Bilind Hajer 27 bilind07@gmail.com,bhajer3@gatech.edu


In [41]:
nlp_toolkit = English()

In [42]:
parsed = nlp_toolkit(sentence_to_unicode) # parse or tokenize the unicode object

In [50]:
entities = parsed.ents # grade entities of tokenized unicode object

In [51]:
def mentions_name(entities, name):
    for entity in entities:
        if entity.text == name:
            return True
    return False

In [58]:
def is_an_action(token):
    if(token.pos == spacy.parts_of_speech.VERB):
        return True
    return False

In [59]:
def get_actions(parsed):
    actions = filter(is_an_action,parsed)
    actions = map(lambda token: token.lemma_ , actions)
    return actions

In [60]:
print mentions_name(entities,"Bilind")

True


In [61]:
print get_actions(parsed)

[u'be', u'like', u'run', u'eat']


In [ ]:
# Write a function that can take a take a sentence parsed by `spacy` and 
# identify if it mentions a company named 'Google'. 
# Remember, `spacy` can find entities and code them `ORG` if they are a company.

In [ ]:
# Write a function that can take a sentence parsed by `spacy` 
# and return the verbs of the sentence (preferably lemmatized)

In [ ]:
# Write a function that identifies countries - HINT: the entity label for 
# countries is GPE (or GeoPolitical Entity)

In [ ]:
from gensim.models.word2vec import Word2Vec
if __name__ == '__main__':
    # Loading the tweet data
    tweets = [unicode(tweet, errors='ignore') for tweet in open('../../assets/dataset/captured-tweets.txt', 'r')]

    # Setting up spacy
    nlp_toolkit = English()

    # For each tweet, parse it using `spacy` and print out if the tweet 
    # has 'release' or 'announce' as a verb.
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        if mentions_company(parsed, 'Google'):
            actions = get_actions(parsed)
            if 'release' in actions or 'announce' in actions:
                print(tweet)

    # Solution to 1f
    # Re-run (d) find country tweets that discuss 'Iran' announcing or releasing.
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        if mentions_country(parsed, 'Iran'):
            actions = get_actions(parsed)
            if 'release' in actions or 'announce' in actions:
                print(tweet)

    # First take the collection of tweets and tokenize them using `spacy`

    # I decided to lemmatized the verbs for easier searching and keep symbols
    # and punctuations

    text_split = [[x.text if x.pos != spacy.parts_of_speech.VERB else x.lemma_ 
                    for x in nlp_toolkit(t)] for t in tweets]

    model = Word2Vec(text_split, size=100, window=4, min_count=5, workers=4)

    model.most_similar(positive=['Syria'])

    # Filter tweets down to those that mention 'Iran' or similar entities and 
    # 'war' or similar entities

    # a: Using spacy
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)
        if mentions_country(parsed, 'Iran') or mentions_country(parsed, 'Iraq'): # ... you could add more
            if 'attack' in get_actions(parsed):
                print(tweet)

    # b: using similarity scores
    for tweet in tweets:
        parsed = nlp_toolkit(tweet)

        similarity_to_iran = max([model.similarity('Iran', tok.text) for tok in parsed if tok.text in model.vocab], 0)
        similarity_to_war = max([model.similarity('war', tok.text) for tok in parsed if tok.text in model.vocab], 0)
        if similarity_to_iran > 0.75 and similarity_to_war > 0.75:
            print(similarity_to_iran, similarity_to_war, tweet)
